# Import des modules

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from ast import literal_eval
import csv

# Création des variables

In [2]:
DF0 = pd.read_csv("./../Tables/ReferentielNoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("NomNormalisé")
DF0

,Types,FormesNonNormalisées,RenvoiToponymique
NomNormalisé,,,
Ackerfuss,['cognominal'],"[Ackenfüs, Ackerfuss, Ackenfͦuͦs, Ackenfuss]",NaN
Adam,['patronymic'],[Adam],NaN
Albrecht,['patronymic'],"[Albreht, Albrecht]",NaN
Aler,[],[Aler],NaN
Allewelt,['cognominal'],[Allewelt],NaN
...,...,...,...
Zirtzer,[],"[Zirzer, Zirtzer]",NaN
Zoller,['occupational'],[Zoller],NaN
"Zoller, zum",['toponymic'],"[züm Zoller, Zoller, zum]",NaN


# Création de la soupe

In [3]:
soup = BeautifulSoup(open('./../Sources/EditionsXML/aves_4R_p131-220.xml', 'r'))

# Création de la liste des éléments de la soupe à comparer

In [4]:
ListeNomsDoc = []
for surname in soup.find_all('surname'):
    ListeNomsDoc.append(surname.get_text())
for addName in soup.find_all('addname'):
    ListeNomsDoc.append(addName.get_text())
print (ListeNomsDoc)

['von Kageneck', 'von Mulnheim', 'Löselin', 'Klobelouch', 'Metziger', 'Marsilius', 'von Mülnheim', 'von Endingen', 'Hüffelin', 'Zorn', 'von Mülnheim', 'Hüffelin', 'Zorn', 'Bock', 'Hesse', 'Merswin', 'Tütschman', 'Rippelin', 'von Heiligenstein', 'Gürteler', 'Bühssener', 'züm Trübel', 'Sturm', 'züm Störcken', 'Göudertheim', 'Völtsche', 'von Wintertur', 'Rebstock', 'Nope', 'von Colmar', 'Gerung', 'Rotschilt', 'Spilman', 'Schott', 'Isenhusen', 'Kindelin', 'von Ginebreht', 'Boppe', 'Tumeler', 'Westerman', 'Heischer', 'Loubeck', 'Honowe', 'Hünrer', 'Nattener', 'Swicker', 'Aler', 'Gosse', 'von Ingenheim', 'Diether', 'Dingesheim', 'von Hagenöwe', 'Stahel', 'Klye', 'Rynöwe', 'von Wickersheim', 'Bock', 'von Heiligenstein', 'von Rosheim', 'Sünner', 'Zorn', 'von Mülnheim', 'Swarber', 'Burggraf', 'Wetzel', 'von Mülnheim', 'Maler', 'von Matzenheim', 'von Stille', 'von Endingen', 'zür Megde', 'Berlin', 'Gürtler', 'Blümenowe', 'Rebstock', 'von Cünheim', 'Mansse', 'Bock', 'Hesse', 'Kloblouch', 'Ellehar

# Comparaison de la liste de la soupe à notre dataframe

In [5]:
for element in ListeNomsDoc :
    
    #Création des listes de références à partir du dataframe DF1.
    Colonne_FormesNonNormalisées = DF0['FormesNonNormalisées'].tolist()
    Colonne_NomNormalisé = DF0.index.tolist()
    Colonne_Types = DF0['Types'].tolist()
    
    #Vérification de l'existence de la forme non normalisée dans la liste des listes de formes non-normalisées.
    CheckSurnameNN = any(element in sublist for sublist in Colonne_FormesNonNormalisées)
    DicoPersonne = {}

    #Sinon, on demande un str en entrée pour le comparer à la liiste des formes normalisées.
    if CheckSurnameNN != True :
        SurnameN = input('Entrez la forme normalisée du nom ' + element + ' :')
        
        #Si l'entrée figure dans la liste des prénoms normalisées, on ajoute la forme non normalisée à la liste de celles-ci dans le dataframe.
        if SurnameN in Colonne_NomNormalisé:
            CelluleModif = DF0.loc[SurnameN, 'FormesNonNormalisées']
            CelluleModif.append(element)
            
        #Sinon, on crée un dataframe qu'on additionnera à l'existant.
        else :
            
            #On crée le couple 'PrénomNormalisé' dont la valeur correspond à la forme la plus courante du prénom.
            DicoPersonne['NomNormalisé'] = SurnameN
            
            #On crée le couple 'Types' dont la valeur est une liste contenant des tags proposant une description typologique du prénom.
            typologie = []
            n = int(input('Entrez le nombre de types à attribuer au nom ' + element + ' :'))
            for i in range(0,n):
                Type = input('Entrez un des types à attribuer au nom ' + element + ' :')
                typologie.append(Type)
            DicoPersonne['Types'] = typologie
            
            #On crée le couple 'FormesNonNormalisées' dont la valeur est une liste contenant toutes les formes alternatives du prénom.
            ListeNN = []
            ListeNN.append(element)
            if element != SurnameN :
                ListeNN.append(SurnameN)
            DicoPersonne['FormesNonNormalisées'] = ListeNN
            
            #On crée le couple 'RenvoiToponymique' dont la valeur est le nom normalisé en français (si possible) du lieu.
            for Type in typologie:
                if Type == 'toponymic':
                    RenvoiToponymique = input('Entrez le nom normalisé, en français si possible, du lieu auquel' + element + ' fait référence : ')
                    DicoPersonne['RenvoiToponymique'] = RenvoiToponymique
            
            #On crée le dataframe à ajouter à l'existant.
            DF1 = pd.DataFrame([DicoPersonne]).set_index('NomNormalisé')
            
            #On concatène le dictionnaire obtenu à notre Referentiel avant de recommencer toute l'opération.
            DF0 = pd.concat([DF0,DF1])
DF0

Entrez la forme normalisée du nom Gerung :Gerung
Entrez le nombre de types à attribuer au nom Gerung :0
Entrez la forme normalisée du nom  :
Entrez le nombre de types à attribuer au nom  :0


,Types,FormesNonNormalisées,RenvoiToponymique
NomNormalisé,,,
Ackerfuss,['cognominal'],"[Ackenfüs, Ackerfuss, Ackenfͦuͦs, Ackenfuss]",NaN
Adam,['patronymic'],[Adam],NaN
Albrecht,['patronymic'],"[Albreht, Albrecht]",NaN
Aler,[],[Aler],NaN
Allewelt,['cognominal'],[Allewelt],NaN
...,...,...,...
"Zoller, zum",['toponymic'],"[züm Zoller, Zoller, zum]",NaN
Zorn,['cognominal'],"[Zorn, zorn]",NaN
"Zweibrücken, von",['toponymic'],"[von Zweibrùcken, Zweibrücken, von]",Zweibrücken


In [8]:
DF0 = DF0.sort_values(by=['NomNormalisé'])
DF0.to_csv("./../Tables/ReferentielNoms.csv")